In [1]:
import math

import random

import os

import rasterio
import rasterio.plot
from rasterio.mask import mask 
from rasterio.plot import plotting_extent

from osgeo import gdal, osr
gdal.UseExceptions()
    
import shapely
import geopandas as gpd
import numpy as np

import matplotlib
import matplotlib.pyplot as plt

import earthpy as et
import earthpy.spatial as es
import earthpy.plot as ep

import errno
import csv
from PIL import Image

### Abrir arquivos

In [2]:
# Abrir arquivo de copas
copas = gpd.read_file('../VECTOR/copas_final.gpkg').set_index('id').sort_index()
copas = copas.loc[copas['valid'] == 1]

img = gdal.Open('../IMGS/odm_orthophoto_bg.tif')

### Gerar conjunto de imagens - foreground

In [ ]:
#######################
# Corte com gdal.Warp #
#######################

for i in copas.index:
    
    out_cut = '../IMGS/sythetic_dataset/cut.gpkg'
    copas.loc[[i]].to_file(out_cut, driver="GPKG")
    
    gdal.Warp('../IMGS/sythetic_dataset/foregrounds/copa_{i}.tif'.format(i=i), 
              img, 
              cutlineDSName=out_cut,
              xRes = img.GetGeoTransform()[1],
              yRes = img.GetGeoTransform()[-1],
#               width=256,
#               height=256,
              targetAlignedPixels=True,
              cropToCutline=True)

### Gerar conjunto de imagens - background

In [ ]:
##############################
## Corte com gdal.Translate ##
## ------------------------ ##
##     Sem grade prévia     ##
##############################

# Acessar EPSG do mosaico
prj = img.GetProjection()
srs = osr.SpatialReference(wkt=prj)
epsg = 'EPSG:' + srs.GetAuthorityCode('projcs')

# Tamanho das imagens (px)
patch_size = 512

# Start do ID e do dicionario de poligonos formadores da grade
i = 1
grid_pol = {'id':[], 'geometry':[]}

# Iterar em cada linha da imagem com passos de mesmo tamanho dos patches
for px_x in range(0, img.RasterXSize, patch_size):
    
    # Iterar em cada coluna da imagem com passos de mesmo tamanho dos patches
    for px_y in range(0, img.RasterYSize, patch_size):
        
        # Gerar o dataset do patch na memoria
        ds = gdal.Translate('/vsimem/corte_{i}.tif'.format(i=i), 
                       img, 
                       srcWin = [px_x, px_y, patch_size, patch_size])
        
        # Gerar poligono envolvente da imagem
        xmin, xpixel, _, ymax, _, ypixel = ds.GetGeoTransform()
        width, height = ds.RasterXSize, ds.RasterYSize
        xmax = xmin + width * xpixel
        ymin = ymax + height * ypixel
        geom = shapely.geometry.box(xmin, ymin, xmax, ymax)
        
        
        # Avaliar se o patch corresponde a uma região da imagem com valores válidos (Não zero) e
        # se o patch apresenta alguma copa no seu interior
        if len(ds.ReadAsArray().nonzero()[0]) != 0 and copas.intersects(geom).any():
            
            # Salvar o pacth em disco          
            gdal.Translate('../IMGS/sythetic_dataset/backgrounds/corte_{i}.tif'.format(i=i), 
                       img, 
                       srcWin = [px_x, px_y, patch_size, patch_size])
            
            # Inserir poligono na lista
            grid_pol['id'].append(i)
            grid_pol['geometry'].append(geom)
            
            # Atualizar identificador
            i += 1

# Gerar o GeoDataFrame com a grade e salvá-la            
grid = gpd.GeoDataFrame(grid_pol, crs=epsg).set_index('id')
grid.to_file('../VECTOR/grid_from_img-sythetic.geojson', driver='GeoJSON')

## Transformar em jpg

### Background

In [ ]:
root = '../IMGS/synthetic_dataset/backgrounds'

for files_ext in os.listdir(root):
    
    file = os.path.splitext(files_ext)[0]

    if os.path.splitext(files_ext)[1].lower() == ".tif":

        outpath = os.path.join(root, 'JPG', file + '.jpg')

        if os.path.isfile(outpath):
            print('A jpeg file already exists for {}'.format(file))

        # If a jpeg is *NOT* present, create one from the tiff.
        else:
            try:
                im = Image.open(os.path.join(root, files_ext))
                print("Generating jpeg for {}".format(file))
#                 im.thumbnail(im.size)
                im.convert('RGBA').save(outpath, "JPG", quality=100)
            except Exception as e:
                print(e)

### Foreground

In [ ]:
root = '../IMGS/synthetic_dataset/foregrounds'

for files_ext in os.listdir(root):
    
    file = os.path.splitext(files_ext)[0]

    if os.path.splitext(files_ext)[1].lower() == ".tif":

        outpath = os.path.join(root, 'PNG', file + '.png')

        if os.path.isfile(outpath):
            print('A jpeg file already exists for {}'.format(file))

        # If a jpeg is *NOT* present, create one from the tiff.
        else:
            try:
                im = Image.open(os.path.join(root, files_ext))
                print("Generating png for {}".format(file))
#                 im.thumbnail(im.size)
                im.save(outpath, "PNG", quality=100)
            except Exception as e:
                print(e)

## Gerar imagens sintéticas

In [4]:
def compose_images(foreground_path, background_path):
    # Make sure the foreground path is valid and open the image
    assert os.path.exists(foreground_path), 'image path does not exist: {}'.format(foreground_path)
    foreground = Image.open(foreground_path)
    foreground_alpha = np.array(foreground.getchannel(3))
    assert np.any(foreground_alpha == 0), 'foreground needs to have some transparency: {}'.format(foreground_path)
    
    # Make sure the background path is valid and open the image
    assert os.path.exists(background_path), 'image path does not exist: {}'.format(background_path)
    assert os.path.splitext(background_path)[1].lower() in ['.png', '.jpg', '.jpeg'], \
        'foreground must be a .png or .jpg file: {}'.format(foreground_path)
    background = Image.open(background_path)
    background = background.convert('RGBA')
    
    # Rotate the foreground
    angle_degrees = random.randint(0, 359)
    foreground = foreground.rotate(angle_degrees, resample=Image.BICUBIC, expand=True)
    
    # Scale the foreground
    scale = random.random() * .5 + .5 # Pick something between .5 and 1
    new_size = (int(foreground.size[0] * scale), int(foreground.size[1] * scale))
    foreground = foreground.resize(new_size, resample=Image.BICUBIC)
    
    # Add any other transformations here...
    
    # Choose a random x,y position for the foreground
    max_xy_position = (background.size[0] - foreground.size[0], background.size[1] - foreground.size[1])
    assert max_xy_position[0] >= 0 and max_xy_position[1] >= 0, \
        'foreground {} is to big for the background {}'.format(foreground_path, background_path)
    paste_position = (random.randint(0, max_xy_position[0]), random.randint(0, max_xy_position[1]))
    
    # Create a new foreground image as large as the background and paste it on top
    new_foreground = Image.new('RGBA', background.size, color = (0, 0, 0, 0))
    new_foreground.paste(foreground, paste_position)
        
    # Extract the alpha channel from the foreground and paste it into a new image the size of the background
    alpha_mask = foreground.getchannel(3)
    new_alpha_mask = Image.new('L', background.size, color=0)
    new_alpha_mask.paste(alpha_mask, paste_position)
    composite = Image.composite(new_foreground, background, new_alpha_mask)
    
    # Grab the alpha pixels above a specified threshold
    alpha_threshold = 200
    mask_arr = np.array(np.greater(np.array(new_alpha_mask), alpha_threshold), dtype=np.uint8)
    hard_mask = Image.fromarray(np.uint8(mask_arr) * 255, 'L')
    
    # Get the smallest & largest non-zero values in each dimension and calculate the bounding box
    nz = np.nonzero(hard_mask)
    bbox = [np.min(nz[0]), np.min(nz[1]), np.max(nz[0]), np.max(nz[1])] 

    return composite, hard_mask, bbox

In [6]:
# Get lists of foreground and background image paths
dataset_dir = '../IMGS/synthetic_dataset'
backgrounds_dir = os.path.join(dataset_dir, 'backgrounds', 'JPG')
foregrounds_dir = os.path.join(dataset_dir, 'foregrounds', 'PNG')
backgrounds = [os.path.join(backgrounds_dir, file_name) for file_name in os.listdir(backgrounds_dir)]
foregrounds = [os.path.join(foregrounds_dir, file_name) for file_name in os.listdir(foregrounds_dir)]

# Create an output directory
output_dir = os.path.join(dataset_dir, 'generated')
try:
    os.mkdir(output_dir)
except OSError as exc:
    if exc.errno != errno.EEXIST:
        raise
    pass

# Create a list to keep track of images and mask annotations
csv_lines = []

# Generate 5 new images
for i in range(1):
    foreground_path = random.choice(foregrounds)
    background_path = random.choice(backgrounds)
    composite, mask, bbox = compose_images(foreground_path, background_path)
    
    composite_path = os.path.join(output_dir, 'image_{0:04d}.png'.format(i))
    composite.save(composite_path)
    
    mask_path = os.path.join(output_dir, 'mask_{0:04d}.png'.format(i))
    mask.save(mask_path)
    
    csv_lines.append([composite_path, bbox, mask_path])
    
# Output the annotations csv
annotations_csv_path = os.path.join(output_dir, 'annotations.csv')
with open(annotations_csv_path, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for csv_line in csv_lines:
        writer.writerow(csv_line)